<a href="https://colab.research.google.com/github/pengwork/ANIML/blob/master/learn2learn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
!pip install learn2learn

In [0]:
import learn2learn as l2l
import torchvision

mnist = torchvision.datasets.MNIST(root="/tmp/mnist", train=True)

mnist = l2l.data.MetaDataset(mnist)
train_tasks = l2l.data.TaskDataset(mnist,
                                   task_transforms=[
                                        NWays(mnist, n=3),
                                        KShots(mnist, k=1),
                                        LoadData(mnist),
                                   ],
                                   num_tasks=10)
model = Net()
maml = l2l.algorithms.MAML(model, lr=1e-3, first_order=False)
opt = optim.Adam(maml.parameters(), lr=4e-3)

for iteration in range(num_iterations):
    learner = maml.clone()  # Creates a clone of model
    for task in train_tasks:
        # Split task in adaptation_task and evalutation_task
        # Fast adapt
        for step in range(adaptation_steps):
            error = compute_loss(adaptation_task)
            learner.adapt(error)

        # Compute evaluation loss
        evaluation_error = compute_loss(evaluation_task)

        # Meta-update the model parameters
        opt.zero_grad()
        evaluation_error.backward()
        opt.step()

NameError: ignored

In [0]:
!pip install fairseq.data.token_block_utils_fast

ERROR: Could not find a version that satisfies the requirement fairseq.data.token_block_utils_fast (from versions: none)
ERROR: No matching distribution found for fairseq.data.token_block_utils_fast


In [0]:
import fairseq
import fairseq.data.token_block_utils_fast

ModuleNotFoundError: ignored

In [0]:
import argparse
import random

import torch
from torch import nn, optim
from torch.nn import functional as F
from tqdm import tqdm

import learn2learn as l2l


class Net(nn.Module):
    """Head for sentence-level classification tasks."""

    def __init__(self, num_classes, input_dim=768, inner_dim=200, pooler_dropout=0.3):
        super().__init__()
        self.dense = nn.Linear(input_dim, inner_dim)
        self.activation_fn = nn.ReLU()
        self.dropout = nn.Dropout(p=pooler_dropout)
        self.out_proj = nn.Linear(inner_dim, num_classes)

    def forward(self, x, **kwargs):
        x = self.dropout(x)
        x = self.dense(x)
        x = self.activation_fn(x)
        x = self.dropout(x)
        x = F.log_softmax(self.out_proj(x), dim=1)
        return x


def accuracy(predictions, targets):
    predictions = predictions.argmax(dim=1)
    acc = (predictions == targets).sum().float()
    acc /= len(targets)
    return acc.item()


def collate_tokens(values, pad_idx, eos_idx=None, left_pad=False, move_eos_to_beginning=False):
    """Convert a list of 1d tensors into a padded 2d tensor."""
    size = max(v.size(0) for v in values)
    res = values[0].new(len(values), size).fill_(pad_idx)

    def copy_tensor(src, dst):
        assert dst.numel() == src.numel()
        if move_eos_to_beginning:
            assert src[-1] == eos_idx
            dst[0] = eos_idx
            dst[1:] = src[:-1]
        else:
            dst.copy_(src)

    for i, v in enumerate(values):
        copy_tensor(v, res[i][size - len(v):] if left_pad else res[i][:len(v)])
    return res


def compute_loss(task, roberta, device, learner, loss_func, batch=15):
    loss = 0.0
    acc = 0.0
    for i, (x, y) in enumerate(torch.utils.data.DataLoader(
            task, batch_size=batch, shuffle=True, num_workers=0)):
        # RoBERTa ENCODING
        x = collate_tokens([roberta.encode(sent) for sent in x], pad_idx=1)
        with torch.no_grad():
            x = roberta.extract_features(x)
        x = x[:, 0, :]

        # Moving to device
        x, y = x.to(device), y.view(-1).to(device)

        output = learner(x)
        curr_loss = loss_func(output, y)
        acc += accuracy(output, y)
        loss += curr_loss / len(task)
    loss /= len(task)
    return loss, acc


def main(lr=0.005, maml_lr=0.01, iterations=1000, ways=5, shots=1, tps=32, fas=5, device=torch.device("cpu"),
         download_location="/tmp/text"):
    text_train = l2l.text.datasets.NewsClassification(root=download_location)
    train_gen = l2l.data.TaskDataset(text_train)

    torch.hub.set_dir(download_location)
    roberta = torch.hub.load('pytorch/fairseq', 'roberta.base')
    roberta.eval()
    roberta.to(device)
    model = Net(num_classes=ways)
    model.to(device)
    meta_model = l2l.algorithms.MAML(model, lr=maml_lr)
    opt = optim.Adam(meta_model.parameters(), lr=lr)
    loss_func = nn.NLLLoss(reduction="sum")

    tqdm_bar = tqdm(range(iterations))
    for iteration in tqdm_bar:
        iteration_error = 0.0
        iteration_acc = 0.0
        for _ in range(tps):
            learner = meta_model.clone()
            train_task = train_gen.sample(shots=shots)
            valid_task = train_gen.sample(shots=shots, classes_to_sample=train_task.sampled_classes)

            # Fast Adaptation
            for step in range(fas):
                train_error, _ = compute_loss(train_task, roberta, device, learner, loss_func, batch=shots * ways)
                learner.adapt(train_error)

            # Compute validation loss
            valid_error, valid_acc = compute_loss(valid_task, roberta, device, learner, loss_func,
                                                  batch=shots * ways)
            iteration_error += valid_error
            iteration_acc += valid_acc

        iteration_error /= tps
        iteration_acc /= tps
        tqdm_bar.set_description("Loss : {:.3f} Acc : {:.3f}".format(iteration_error.item(), iteration_acc))

        # Take the meta-learning step
        opt.zero_grad()
        iteration_error.backward()
        opt.step()


if __name__ == '__main__':
  no_cuda = False
  use_cuda = not no_cuda and torch.cuda.is_available()

  torch.manual_seed(1)
  random.seed(1)

  device = torch.device("cuda" if use_cuda else "cpu")

  main(lr=0.005, maml_lr=0.01, iterations=1000, ways=5, shots=1, tps=32, fas=5, device=device,download_location='temp/text')




    # parser = argparse.ArgumentParser(description='Learn2Learn Text Classification Example')

    # parser.add_argument('--ways', type=int, default=5, metavar='N',
    #                     help='number of ways (default: 5)')
    # parser.add_argument('--shots', type=int, default=1, metavar='N',
    #                     help='number of shots (default: 1)')
    # parser.add_argument('-tps', '--tasks-per-step', type=int, default=32, metavar='N',
    #                     help='tasks per step (default: 32)')
    # parser.add_argument('-fas', '--fast-adaption-steps', type=int, default=5, metavar='N',
    #                     help='steps per fast adaption (default: 5)')

    # parser.add_argument('--iterations', type=int, default=1000, metavar='N',
    #                     help='number of iterations (default: 1000)')

    # parser.add_argument('--lr', type=float, default=0.005, metavar='LR',
    #                     help='learning rate (default: 0.005)')
    # parser.add_argument('--maml-lr', type=float, default=0.01, metavar='LR',
    #                     help='learning rate for MAML (default: 0.01)')

    # parser.add_argument('--no-cuda', action='store_true', default=False,
    #                     help='disables CUDA training')

    # parser.add_argument('--seed', type=int, default=1, metavar='S',
    #                     help='random seed (default: 1)')

    # parser.add_argument('--download-location', type=str, default="/tmp/text", metavar='S',
    #                     help='download location for train data and roberta(default : /tmp/text')

    # args = parser.parse_args()

    # use_cuda = not args.no_cuda and torch.cuda.is_available()

    # torch.manual_seed(args.seed)
    # random.seed(args.seed)

    # device = torch.device("cuda" if use_cuda else "cpu")

    # main(lr=args.lr, maml_lr=args.maml_lr, iterations=args.iterations, ways=args.ways, shots=args.shots,
    #      tps=args.tasks_per_step, fas=args.fast_adaption_steps, device=device,
    #      download_location=args.download_location)

Using cache found in temp/text/pytorch_fairseq_master


FileNotFoundError: ignored

In [0]:
ls

drive/  pytorch/  sample_data/  temp/


In [0]:
!python setup.py install

In [0]:
import os
print(os.getcwd())
os.chdir("/content/temp/text/pytorch_fairseq_master")
print(os.getcwd())
# path = "/content/gdrive/My Drive/sample"
# os.chdir(path)
# 返回上一级目录
# os.chdir('../')
# print(os.getcwd())

/content
/content/temp/text/pytorch_fairseq_master


In [0]:
import os
print(os.getcwd())
os.chdir("/content")
print(os.getcwd())
# path = "/content/gdrive/My Drive/sample"
# os.chdir(path)
# 返回上一级目录
# os.chdir('../')

/content
/content


In [52]:
!python test.py

Using cache found in /tmp/text/pytorch_fairseq_master
  0% 0/1000 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/torch/tensor.py:746: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  warnings.warn("The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad "
Traceback (most recent call last):
  File "test.py", line 167, in <module>
    download_location=args.download_location)
  File "test.py", line 108, in main
    train_error, _ = compute_loss(train_task, roberta, device, learner, loss_func, batch=shots * ways)
  File "test.py", line 61, in compute_loss
    for i, (x, y) in enumerate(torch.

In [0]:
pip install fairseq

In [45]:
import torch as th
from torch import nn, optim, distributions as dist

import learn2learn as l2l

DIM = 5
TIMESTEPS = 1000
TASKS_PER_STEP = 50


class Model(nn.Module):

    def __init__(self):
        super(Model, self).__init__()
        self.mu = nn.Parameter(th.randn(DIM))
        self.sigma = nn.Parameter(th.randn(DIM))

    def forward(self, x=None):
        return dist.Normal(self.mu, self.sigma)


def main():
    task_dist = dist.Normal(th.zeros(2 * DIM), th.ones(2 * DIM))
    model = Model()
    maml = l2l.algorithms.MAML(model, lr=1e-2)
    opt = optim.Adam(maml.parameters())

    for i in range(TIMESTEPS):
        step_loss = 0.0
        for t in range(TASKS_PER_STEP):
            # Sample a task
            task_params = task_dist.sample()
            mu_i, sigma_i = task_params[:DIM], task_params[DIM:]

            # Adaptation: Instanciate a copy of model
            learner = maml.clone()
            proposal = learner()

            # Adaptation: Compute and adapt to task loss
            loss = (mu_i - proposal.mean).pow(2).sum() + (sigma_i - proposal.variance).pow(2).sum()
            learner.adapt(loss)

            # Adaptation: Evaluate the effectiveness of adaptation
            adapt_loss = (mu_i - proposal.mean).pow(2).sum() + (sigma_i - proposal.variance).pow(2).sum()

            # Accumulate the error over all tasks
            step_loss += adapt_loss

        # Meta-learning step: compute gradient through the adaptation step, automatically.
        step_loss = step_loss / TASKS_PER_STEP
        print(i, step_loss.item())
        opt.zero_grad()
        step_loss.backward()
        opt.step()


if __name__ == '__main__':
    main()

/usr/local/lib/python3.6/dist-packages/torch/tensor.py:746: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  warnings.warn("The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad "


0 17.061962127685547
1 19.113014221191406
2 17.863384246826172
3 18.85416603088379
4 17.07343864440918
5 18.036943435668945
6 15.934466361999512
7 16.920808792114258
8 19.387113571166992
9 19.938034057617188
10 18.060535430908203
11 17.102968215942383
12 17.253414154052734
13 19.59915542602539
14 19.163097381591797
15 17.912517547607422
16 17.48438262939453
17 16.467491149902344
18 17.994991302490234
19 18.183429718017578
20 16.77085304260254
21 16.73499870300293
22 17.567752838134766
23 17.65776824951172
24 17.055801391601562
25 17.486984252929688
26 19.05139923095703
27 17.9143123626709
28 17.68560218811035
29 18.0966739654541
30 17.1558895111084
31 18.50570297241211
32 17.169475555419922
33 18.681337356567383
34 18.175025939941406
35 18.50633430480957
36 17.4508056640625
37 17.4473876953125
38 16.67022132873535
39 17.968456268310547
40 18.133161544799805
41 17.3918399810791
42 16.110803604125977
43 17.38953971862793
44 17.907304763793945
45 17.025665283203125
46 16.11427879333496
47

KeyboardInterrupt: ignored

In [0]:
!pip show

In [0]:
!where pip

/bin/bash: where: command not found


In [0]:
!where python

/bin/bash: where: command not found


In [0]:
!which pip

/usr/local/bin/pip
